In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

In [5]:
pt_train=pd.read_csv('Property_train.csv')
pt_test=pd.read_csv('Property_test_share.csv')

In [6]:
target='Junk'

In [7]:
pt_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62035 entries, 0 to 62034
Data columns (total 31 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Junk                   62035 non-null  int64 
 1   InteriorsStyle         62035 non-null  object
 2   PriceIndex8            62035 non-null  object
 3   ListDate               62035 non-null  object
 4   Material               62035 non-null  object
 5   PriceIndex9            62035 non-null  int64 
 6   Agency                 62035 non-null  object
 7   AreaIncomeType         62035 non-null  object
 8   EnvRating              62035 non-null  object
 9   PriceIndex7            62035 non-null  object
 10  ExpeditedListing       62035 non-null  int64 
 11  PriceIndex4            62035 non-null  object
 12  PriceIndex1            62035 non-null  object
 13  PriceIndex6            62035 non-null  object
 14  PRIMEUNIT              62035 non-null  object
 15  Channel            

In [8]:
cat_to_numeric=['PriceIndex'+str(i) for i in range(1,10)]

In [9]:
pt_train['data']='train'
pt_test['data']='test'

all_data=pd.concat([pt_train,pt_test],0,sort=False)

In [10]:
for col in cat_to_numeric:
    all_data[col]=pd.to_numeric(all_data[col],errors='coerce')
    all_data[col]=all_data[col].fillna(all_data.loc[all_data['data']=='train',col].median())

In [11]:
cat_cols=['InteriorsStyle', 'ListDate', 'Material', 'Agency', 'AreaIncomeType',
       'EnvRating', 'PRIMEUNIT', 'Channel', 'PlotType', 'Architecture',
       'Region', 'SubModel', 'Facade', 'State', 'RegionType','Zip']

In [12]:
for col in cat_cols:
    k=all_data[col].value_counts()
    cats=k[k>=1000].index[:-1]
    
    for cat in cats:
        name=col+'_'+str(cat)
        all_data[name]=(all_data[col]==cat).astype(int)
        
    del all_data[col]

In [13]:
all_data.shape

(72983, 125)

In [14]:
x_train=all_data.drop([target,'data'],1)[all_data['data']=='train']
y_train=all_data[target][all_data['data']=='train']
x_test=all_data.drop([target,'data'],1)[all_data['data']=='test']

In [15]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 72983 entries, 0 to 10947
Columns: 125 entries, Junk to Zip_32124
dtypes: float64(9), int32(109), int64(6), object(1)
memory usage: 39.8+ MB


In [16]:
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from sklearn import tree

In [17]:
dtree=tree.DecisionTreeClassifier(criterion="entropy",max_leaf_nodes=20,
                                  class_weight="balanced")

In [18]:
dtree.fit(x_train,y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight='balanced',
                       criterion='entropy', max_depth=None, max_features=None,
                       max_leaf_nodes=20, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       presort='deprecated', random_state=None,
                       splitter='best')

In [19]:
from sklearn.metrics import roc_auc_score

In [20]:
roc_auc_score(y_train,dtree.predict(x_train))

0.6726133727715479

In [21]:
submissions=pd.DataFrame({'Junk':dtree.predict_proba(x_test)[:,1]})

In [22]:
submissions.to_csv(r'C:\Users\Prabhakar\Downloads\\sample_submission1.csv', index=False)